In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('results_table_list.csv')

In [3]:
df.columns

Index(['Student_id', 'Student_Test_id', 'Result_id', 'answer', 'graph_id',
       'answer_text', 'question_id', 'question', 'Question_type', 'dataset'],
      dtype='object')

In [4]:
new_df = pd.DataFrame()
for ix, data in df.groupby(['Student_id', 'Question_type', 'dataset',
                            'question_id']):
    if ix[1] == 'pre_test':
        if len(data)>3:
            new_df = new_df.append(data.iloc[:3])
        else:
            new_df = new_df.append(data)
    else:
        if len(data)>1:
            new_df = new_df.append(data.iloc[0])
        else:
            new_df = new_df.append(data)

In [5]:
final_df = pd.DataFrame()
new_question_text = ['(First) Each of these three visualizations cover the same data. Please rank them in the order you think reflects their overall effectiveness and design quality.',
                    '(Second) Each of these three visualizations cover the same data. Please rank them in the order you think reflects their overall effectiveness and design quality.',
                    '(Third) Each of these three visualizations cover the same data. Please rank them in the order you think reflects their overall effectiveness and design quality.']
for ix, data in new_df.groupby(['Student_id', 'Question_type', 'dataset']):

    if ix[1] == 'pre_test':
        data.answer_text = data.graph_id
        data.question = new_question_text
        final_df = final_df.append(data)
    elif ix[1] == 'rating':
        data.answer_text = data.answer
        final_df = final_df.append(data)
    else:
        final_df = final_df.append(data)

/Users/colingerber/Documents/Marti_CS160_Project/env/lib/python3.4/site-packages/pandas/core/generic.py:2177: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [6]:
#add condition data
df_sid = pd.read_csv('student_id_list.csv')
df_sid.Questions = df_sid.Questions.apply(lambda x: int(x))
df_sid.Heuristics = df_sid.Heuristics.apply(lambda x: int(x))
question_student_id_list = [int(x) for x in list(df_sid.Questions)]
heuristic_student_id_list = [int(x) for x in list(df_sid.Heuristics)]

condition_list = []
for ix,data in final_df.iterrows():
    if data.Student_id in question_student_id_list:
        condition_list.append('question')
    elif data.Student_id in heuristic_student_id_list:
        condition_list.append('heuristic')

In [7]:
final_df.condition = condition_list

In [8]:
final_df.head()

,Student_id,Student_Test_id,Result_id,answer,graph_id,answer_text,question_id,question,Question_type,dataset
759,11814,549,907,63,1,neutral,6,This visualization uses visual components appr...,heuristic,0
761,11814,550,909,72,1,disagree,7,This visualization makes important information...,heuristic,0
767,11814,551,917,83,1,neutral,8,This visualization successfully presents multi...,heuristic,0
606,11814,541,744,262,22,disagree,26,This visualization uses visual components appr...,heuristic,2
747,11814,542,891,273,22,neutral,27,This visualization makes important information...,heuristic,2


In [9]:
final_df.to_csv('results_table_cleaned.csv', index=False)